In [207]:
# libraries to use
import requests
import pandas as pd
import json
import io
import itertools
import datetime

In [2]:
# use these headers for every request to NBA.com
headers  = {
    'Connection': 'keep-alive',
    'Accept': 'application/json, text/plain, */*',
    'x-nba-stats-token': 'true',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
    'x-nba-stats-origin': 'stats',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Referer': 'https://stats.nba.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
}

In [3]:
# get general (Measure=Base) player total stats (PerMode=Totals) for 2022-23 season (Season=2022-23) - example URL request
players_stats_url = 'https://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Totals&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=2022-23&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision=&Weight='
players_stats_response = requests.get(url=players_stats_url, headers=headers).json()
players_stats_headers = players_stats_response['resultSets'][0]['headers']
players_stats_data = players_stats_response['resultSets'][0]['rowSet']
players_stats_df = pd.DataFrame(data=players_stats_data, columns=players_stats_headers)

In [4]:
players_stats_headers

['PLAYER_ID',
 'PLAYER_NAME',
 'NICKNAME',
 'TEAM_ID',
 'TEAM_ABBREVIATION',
 'AGE',
 'GP',
 'W',
 'L',
 'W_PCT',
 'MIN',
 'FGM',
 'FGA',
 'FG_PCT',
 'FG3M',
 'FG3A',
 'FG3_PCT',
 'FTM',
 'FTA',
 'FT_PCT',
 'OREB',
 'DREB',
 'REB',
 'AST',
 'TOV',
 'STL',
 'BLK',
 'BLKA',
 'PF',
 'PFD',
 'PTS',
 'PLUS_MINUS',
 'NBA_FANTASY_PTS',
 'DD2',
 'TD3',
 'WNBA_FANTASY_PTS',
 'GP_RANK',
 'W_RANK',
 'L_RANK',
 'W_PCT_RANK',
 'MIN_RANK',
 'FGM_RANK',
 'FGA_RANK',
 'FG_PCT_RANK',
 'FG3M_RANK',
 'FG3A_RANK',
 'FG3_PCT_RANK',
 'FTM_RANK',
 'FTA_RANK',
 'FT_PCT_RANK',
 'OREB_RANK',
 'DREB_RANK',
 'REB_RANK',
 'AST_RANK',
 'TOV_RANK',
 'STL_RANK',
 'BLK_RANK',
 'BLKA_RANK',
 'PF_RANK',
 'PFD_RANK',
 'PTS_RANK',
 'PLUS_MINUS_RANK',
 'NBA_FANTASY_PTS_RANK',
 'DD2_RANK',
 'TD3_RANK',
 'WNBA_FANTASY_PTS_RANK']

In [5]:
players_stats_df.head()

,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,...,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK
0,1630639,A.J. Lawson,A.J.,1610612742,DAL,22.0,13,5,8,0.385,...,460,76,64,447,435,290,449,220,32,447
1,1631260,AJ Green,AJ,1610612749,MIL,23.0,27,21,6,0.778,...,460,47,120,447,364,338,394,220,32,386
2,1631100,AJ Griffin,AJ,1610612737,ATL,19.0,57,27,30,0.474,...,304,320,255,309,161,233,208,220,32,201
3,203932,Aaron Gordon,Aaron,1610612743,DEN,27.0,50,36,14,0.720,...,55,478,347,55,74,3,71,49,32,75
4,1628988,Aaron Holiday,Aaron,1610612737,ATL,26.0,51,27,24,0.529,...,258,294,246,250,303,266,312,220,32,311


In [175]:
# get NBA.com player IDs for the 2022-23 season to loop through later
player_ids_2022_23 = players_stats_df[['PLAYER_ID', 'PLAYER_NAME','GP','MIN']]
player_ids_2022_23

,PLAYER_ID,PLAYER_NAME,GP,MIN
0,1630639,A.J. Lawson,13,62.668333
1,1631260,AJ Green,27,263.321667
2,1631100,AJ Griffin,57,1152.518333
3,203932,Aaron Gordon,50,1511.790000
4,1628988,Aaron Holiday,51,727.113333
...,...,...,...,...
505,1628380,Zach Collins,50,1068.821667
506,203897,Zach LaVine,57,2043.328333
507,1630192,Zeke Nnaji,44,583.638333
508,1630533,Ziaire Williams,29,490.325000


In [316]:
mpg_list = []

for x in range(0, player_ids_2022_23.shape[0]):
    mpg = player_ids_2022_23.iloc[x]['MIN']/player_ids_2022_23.iloc[x]['GP']
    mpg_list.append(round(mpg,1))

player_ids_2022_23.loc[:,'MPG'] = mpg_list

C:\Users\Reed\AppData\Local\Temp\ipykernel_7540\4221380314.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_ids_2022_23.loc[:,'MPG'] = mpg_list


In [317]:
player_ids_2022_23

,PLAYER_ID,PLAYER_NAME,GP,MIN,MPG
0,1630639,A.J. Lawson,13,62.668333,4.8
1,1631260,AJ Green,27,263.321667,9.8
2,1631100,AJ Griffin,57,1152.518333,20.2
3,203932,Aaron Gordon,50,1511.790000,30.2
4,1628988,Aaron Holiday,51,727.113333,14.3
...,...,...,...,...,...
505,1628380,Zach Collins,50,1068.821667,21.4
506,203897,Zach LaVine,57,2043.328333,35.8
507,1630192,Zeke Nnaji,44,583.638333,13.3
508,1630533,Ziaire Williams,29,490.325000,16.9


In [193]:
# get players with MIN > 500 and MPG > 24
high_mpg_players_2022_23_df = player_ids_2022_23.loc[(player_ids_2022_23['MIN'] >= 500) & (player_ids_2022_23['MPG'] >= 24)]

In [194]:
high_mpg_players_2022_23_df

,PLAYER_ID,PLAYER_NAME,GP,MIN,MPG
3,203932,Aaron Gordon,50,1511.790000,30.2
5,1630174,Aaron Nesmith,55,1354.323333,24.6
8,201143,Al Horford,47,1446.805000,30.8
11,1627936,Alex Caruso,51,1236.101667,24.2
15,1630578,Alperen Sengun,55,1547.060000,28.1
...,...,...,...,...,...
486,1630178,Tyrese Maxey,40,1306.841667,32.7
492,203506,Victor Oladipo,28,779.886667,27.9
497,1628976,Wendell Carter Jr.,41,1223.006667,29.8
506,203897,Zach LaVine,57,2043.328333,35.8


In [195]:
high_mpg_players_2022_23_df.to_csv('players_mpg_2022_23.csv', index=False)

In [134]:
# player_id = 2544 # leBron James
# player_id = 202695 # kawhi leoard - lots of time off
player_id = 1629029 # Luka Doncic


In [7]:
# make a list of NBA seasons to loop through later
season_list = [
    '1996-97',
    '1997-98',
    '1998-99',
    '1999-00',
    '2000-01',
    '2001-02',
    '2002-03',
    '2003-04',
    '2004-05',
    '2005-06',
    '2006-07',
    '2007-08',
    '2008-09',
    '2009-10',
    '2010-11',
    '2011-12',
    '2012-13',
    '2013-14',
    '2014-15',
    '2015-16',
    '2016-17',
    '2017-18',
    '2018-19',
    '2019-20',
    '2020-21',
    '2021-22',
    '2022-23',
]

In [8]:
# make a list of NBA measure types to loop through for a player
measure_types = [
    'Base',
    'Advanced',
    'Misc',
    'Scoring',
    'Usage'
]

In [136]:
# example 2022-23 (Season=2022-23) player game logs/box stats (https://stats.nba.com/stats/playergamelogs/playergamelogs): Luka Doncic (PlayerID=1629029) - Basic (MeasureType=Base)
plyr_base_url = 'https://stats.nba.com/stats/playergamelogs?DateFrom=&DateTo=&GameSegment=&LastNGames=0&LeagueID=00&Location=&MeasureType=Base&Month=0&OppTeamID=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Totals&Period=0&PlayerID='+str(player_id)+'&PlusMinus=N&Rank=N&Season=2022-23&SeasonSegment=&SeasonType=Regular%20Season&ShotClockRange=&TeamID=&VsConference=&VsDivision='
plyr_base_response = requests.get(url=plyr_base_url, headers=headers).json()
plyr_base_headers = plyr_base_response['resultSets'][0]['headers']
plyr_base_data = plyr_base_response['resultSets'][0]['rowSet']
plyr_base_df = pd.DataFrame(data=plyr_base_data, columns=plyr_base_headers)

In [137]:
plyr_base_headers

['SEASON_YEAR',
 'PLAYER_ID',
 'PLAYER_NAME',
 'NICKNAME',
 'TEAM_ID',
 'TEAM_ABBREVIATION',
 'TEAM_NAME',
 'GAME_ID',
 'GAME_DATE',
 'MATCHUP',
 'WL',
 'MIN',
 'FGM',
 'FGA',
 'FG_PCT',
 'FG3M',
 'FG3A',
 'FG3_PCT',
 'FTM',
 'FTA',
 'FT_PCT',
 'OREB',
 'DREB',
 'REB',
 'AST',
 'TOV',
 'STL',
 'BLK',
 'BLKA',
 'PF',
 'PFD',
 'PTS',
 'PLUS_MINUS',
 'NBA_FANTASY_PTS',
 'DD2',
 'TD3',
 'WNBA_FANTASY_PTS',
 'GP_RANK',
 'W_RANK',
 'L_RANK',
 'W_PCT_RANK',
 'MIN_RANK',
 'FGM_RANK',
 'FGA_RANK',
 'FG_PCT_RANK',
 'FG3M_RANK',
 'FG3A_RANK',
 'FG3_PCT_RANK',
 'FTM_RANK',
 'FTA_RANK',
 'FT_PCT_RANK',
 'OREB_RANK',
 'DREB_RANK',
 'REB_RANK',
 'AST_RANK',
 'TOV_RANK',
 'STL_RANK',
 'BLK_RANK',
 'BLKA_RANK',
 'PF_RANK',
 'PFD_RANK',
 'PTS_RANK',
 'PLUS_MINUS_RANK',
 'NBA_FANTASY_PTS_RANK',
 'DD2_RANK',
 'TD3_RANK',
 'WNBA_FANTASY_PTS_RANK',
 'VIDEO_AVAILABLE_FLAG']

In [138]:
plyr_base_headers_subset = ['SEASON_YEAR',
 'PLAYER_ID',
 'PLAYER_NAME',
 'TEAM_ID',
 'TEAM_ABBREVIATION',
 'TEAM_NAME',
 'GAME_ID',
 'GAME_DATE',
 'MATCHUP',
 'WL',
 'MIN',
 'FGM',
 'FGA',
 'FG3M',
 'FG3A',
 'FTM',
 'FTA',
 'OREB',
 'DREB',
 'AST',
 'TOV',
 'STL',
 'BLK',
 'PF',
 'PFD',
 'PTS']

In [139]:
plyr_base_df_subset = plyr_base_df[plyr_base_headers_subset] 

In [140]:
plyr_base_df_subset.head()

,SEASON_YEAR,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,...,FTA,OREB,DREB,AST,TOV,STL,BLK,PF,PFD,PTS
0,2022-23,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022200913,2023-02-26T00:00:00,DAL vs. LAL,L,...,4,0,9,5,6,1,0,3,5,26
1,2022-23,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022200891,2023-02-23T00:00:00,DAL vs. SAS,W,...,12,0,7,10,1,2,2,0,7,28
2,2022-23,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022200881,2023-02-15T00:00:00,DAL @ DEN,L,...,12,1,3,9,2,4,0,1,6,37
3,2022-23,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022200865,2023-02-13T00:00:00,DAL vs. MIN,L,...,15,1,11,6,4,1,0,3,10,33
4,2022-23,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022200854,2023-02-11T00:00:00,DAL @ SAC,L,...,5,1,8,5,6,0,0,0,4,27


In [29]:
# example 2022-23 (Season=2022-23) player game logs/box stats (https://stats.nba.com/stats/playergamelogs/playergamelogs): Luka Doncic (PlayerID=1629029) - Advanced (MeasureType=Advanced)
plyr_advanced_url = 'https://stats.nba.com/stats/playergamelogs?DateFrom=&DateTo=&GameSegment=&LastNGames=0&LeagueID=00&Location=&MeasureType=Advanced&Month=0&OppTeamID=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Totals&Period=0&PlayerID='+str(player_id)+'&PlusMinus=N&Rank=N&Season=2022-23&SeasonSegment=&SeasonType=Regular%20Season&ShotClockRange=&TeamID=&VsConference=&VsDivision='
plyr_advanced_response = requests.get(url=plyr_advanced_url, headers=headers).json()
plyr_advanced_headers = plyr_advanced_response['resultSets'][0]['headers']
plyr_advanced_data = plyr_advanced_response['resultSets'][0]['rowSet']
plyr_advanced_df = pd.DataFrame(data=plyr_advanced_data, columns=plyr_advanced_headers)

In [30]:
plyr_advanced_headers

['SEASON_YEAR',
 'PLAYER_ID',
 'PLAYER_NAME',
 'NICKNAME',
 'TEAM_ID',
 'TEAM_ABBREVIATION',
 'TEAM_NAME',
 'GAME_ID',
 'GAME_DATE',
 'MATCHUP',
 'WL',
 'MIN',
 'E_OFF_RATING',
 'OFF_RATING',
 'sp_work_OFF_RATING',
 'E_DEF_RATING',
 'DEF_RATING',
 'sp_work_DEF_RATING',
 'E_NET_RATING',
 'NET_RATING',
 'sp_work_NET_RATING',
 'AST_PCT',
 'AST_TO',
 'AST_RATIO',
 'OREB_PCT',
 'DREB_PCT',
 'REB_PCT',
 'TM_TOV_PCT',
 'E_TOV_PCT',
 'EFG_PCT',
 'TS_PCT',
 'USG_PCT',
 'E_USG_PCT',
 'E_PACE',
 'PACE',
 'PACE_PER40',
 'sp_work_PACE',
 'PIE',
 'POSS',
 'FGM',
 'FGA',
 'FGM_PG',
 'FGA_PG',
 'FG_PCT',
 'GP_RANK',
 'W_RANK',
 'L_RANK',
 'W_PCT_RANK',
 'MIN_RANK',
 'E_OFF_RATING_RANK',
 'OFF_RATING_RANK',
 'sp_work_OFF_RATING_RANK',
 'E_DEF_RATING_RANK',
 'DEF_RATING_RANK',
 'sp_work_DEF_RATING_RANK',
 'E_NET_RATING_RANK',
 'NET_RATING_RANK',
 'sp_work_NET_RATING_RANK',
 'AST_PCT_RANK',
 'AST_TO_RANK',
 'AST_RATIO_RANK',
 'OREB_PCT_RANK',
 'DREB_PCT_RANK',
 'REB_PCT_RANK',
 'TM_TOV_PCT_RANK',
 'E_TOV

In [141]:
plyr_advanced_headers_subset = ['USG_PCT',
 'PIE',
 'POSS']
plyr_advanced_df_subset = plyr_advanced_df[plyr_advanced_headers_subset]
plyr_advanced_df_subset.head()

,USG_PCT,PIE,POSS
0,0.319,0.140,78
1,0.319,0.264,65
2,0.359,0.236,72
3,0.378,0.202,73
4,0.304,0.121,85


In [142]:
# example 2022-23 (Season=2022-23) player game logs/box stats (https://stats.nba.com/stats/playergamelogs/playergamelogs): Luka Doncic (PlayerID=1629029) - Basic (MeasureType=Misc)
plyr_misc_url = 'https://stats.nba.com/stats/playergamelogs?DateFrom=&DateTo=&GameSegment=&LastNGames=0&LeagueID=00&Location=&MeasureType=Misc&Month=0&OppTeamID=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Totals&Period=0&PlayerID='+str(player_id)+'&PlusMinus=N&Rank=N&Season=2022-23&SeasonSegment=&SeasonType=Regular%20Season&ShotClockRange=&TeamID=&VsConference=&VsDivision='
plyr_misc_response = requests.get(url=plyr_misc_url, headers=headers).json()
plyr_misc_headers = plyr_misc_response['resultSets'][0]['headers']
plyr_misc_data = plyr_misc_response['resultSets'][0]['rowSet']
plyr_misc_df = pd.DataFrame(data=plyr_misc_data, columns=plyr_misc_headers)

In [143]:
plyr_misc_headers

['SEASON_YEAR',
 'PLAYER_ID',
 'PLAYER_NAME',
 'NICKNAME',
 'TEAM_ID',
 'TEAM_ABBREVIATION',
 'TEAM_NAME',
 'GAME_ID',
 'GAME_DATE',
 'MATCHUP',
 'WL',
 'MIN',
 'PTS_OFF_TOV',
 'PTS_2ND_CHANCE',
 'PTS_FB',
 'PTS_PAINT',
 'OPP_PTS_OFF_TOV',
 'OPP_PTS_2ND_CHANCE',
 'OPP_PTS_FB',
 'OPP_PTS_PAINT',
 'BLK',
 'BLKA',
 'PF',
 'PFD',
 'NBA_FANTASY_PTS',
 'GP_RANK',
 'W_RANK',
 'L_RANK',
 'W_PCT_RANK',
 'MIN_RANK',
 'PTS_OFF_TOV_RANK',
 'PTS_2ND_CHANCE_RANK',
 'PTS_FB_RANK',
 'PTS_PAINT_RANK',
 'OPP_PTS_OFF_TOV_RANK',
 'OPP_PTS_2ND_CHANCE_RANK',
 'OPP_PTS_FB_RANK',
 'OPP_PTS_PAINT_RANK',
 'BLK_RANK',
 'BLKA_RANK',
 'PF_RANK',
 'PFD_RANK',
 'NBA_FANTASY_PTS_RANK',
 'VIDEO_AVAILABLE_FLAG']

In [144]:
plyr_misc_headers_subset= [
 'PTS_PAINT']
plyr_misc_df_subset = plyr_misc_df[plyr_misc_headers_subset]
plyr_misc_df_subset.head()

,PTS_PAINT
0,10
1,14
2,12
3,16
4,16


In [145]:
# example 2022-23 (Season=2022-23) player game logs/box stats (https://stats.nba.com/stats/playergamelogs/playergamelogs): Luka Doncic (PlayerID=1629029) - Basic (MeasureType=Scoring)
plyr_scor_url = 'https://stats.nba.com/stats/playergamelogs?DateFrom=&DateTo=&GameSegment=&LastNGames=0&LeagueID=00&Location=&MeasureType=Scoring&Month=0&OppTeamID=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Totals&Period=0&PlayerID='+str(player_id)+'&PlusMinus=N&Rank=N&Season=2022-23&SeasonSegment=&SeasonType=Regular%20Season&ShotClockRange=&TeamID=&VsConference=&VsDivision='
plyr_scor_response = requests.get(url=plyr_scor_url, headers=headers).json()
plyr_scor_headers = plyr_scor_response['resultSets'][0]['headers']
plyr_scor_data = plyr_scor_response['resultSets'][0]['rowSet']
plyr_scor_df = pd.DataFrame(data=plyr_scor_data, columns=plyr_scor_headers)

In [146]:
plyr_scor_headers # do we need anything...?

['SEASON_YEAR',
 'PLAYER_ID',
 'PLAYER_NAME',
 'NICKNAME',
 'TEAM_ID',
 'TEAM_ABBREVIATION',
 'TEAM_NAME',
 'GAME_ID',
 'GAME_DATE',
 'MATCHUP',
 'WL',
 'MIN',
 'PCT_FGA_2PT',
 'PCT_FGA_3PT',
 'PCT_PTS_2PT',
 'PCT_PTS_2PT_MR',
 'PCT_PTS_3PT',
 'PCT_PTS_FB',
 'PCT_PTS_FT',
 'PCT_PTS_OFF_TOV',
 'PCT_PTS_PAINT',
 'PCT_AST_2PM',
 'PCT_UAST_2PM',
 'PCT_AST_3PM',
 'PCT_UAST_3PM',
 'PCT_AST_FGM',
 'PCT_UAST_FGM',
 'FGM',
 'FGA',
 'FG_PCT',
 'GP_RANK',
 'W_RANK',
 'L_RANK',
 'W_PCT_RANK',
 'MIN_RANK',
 'PCT_FGA_2PT_RANK',
 'PCT_FGA_3PT_RANK',
 'PCT_PTS_2PT_RANK',
 'PCT_PTS_2PT_MR_RANK',
 'PCT_PTS_3PT_RANK',
 'PCT_PTS_FB_RANK',
 'PCT_PTS_FT_RANK',
 'PCT_PTS_OFF_TOV_RANK',
 'PCT_PTS_PAINT_RANK',
 'PCT_AST_2PM_RANK',
 'PCT_UAST_2PM_RANK',
 'PCT_AST_3PM_RANK',
 'PCT_UAST_3PM_RANK',
 'PCT_AST_FGM_RANK',
 'PCT_UAST_FGM_RANK',
 'FGM_RANK',
 'FGA_RANK',
 'FG_PCT_RANK',
 'VIDEO_AVAILABLE_FLAG']

In [147]:
plyr_scor_headers_subset = ['PCT_PTS_2PT_MR']
plyr_scor_df_subset = plyr_scor_df[plyr_scor_headers_subset]
plyr_scor_df_subset.head()

,PCT_PTS_2PT_MR
0,0.077
1,0.000
2,0.162
3,0.000
4,0.074


In [149]:
# example 2022-23 (Season=2022-23) player game logs/box stats (https://stats.nba.com/stats/playergamelogs/playergamelogs): Luka Doncic (PlayerID=1629029) - Basic (MeasureType=Usage)
plyr_usage_url = 'https://stats.nba.com/stats/playergamelogs?DateFrom=&DateTo=&GameSegment=&LastNGames=0&LeagueID=00&Location=&MeasureType=Usage&Month=0&OppTeamID=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Totals&Period=0&PlayerID='+str(player_id)+'&PlusMinus=N&Rank=N&Season=2022-23&SeasonSegment=&SeasonType=Regular%20Season&ShotClockRange=&TeamID=&VsConference=&VsDivision='
plyr_usage_response = requests.get(url=plyr_usage_url, headers=headers).json()
plyr_usage_headers = plyr_usage_response['resultSets'][0]['headers']
plyr_usage_data = plyr_usage_response['resultSets'][0]['rowSet']
plyr_usage_df = pd.DataFrame(data=plyr_usage_data, columns=plyr_usage_headers)

In [150]:
plyr_usage_headers # do we need anything...?

['SEASON_YEAR',
 'PLAYER_ID',
 'PLAYER_NAME',
 'NICKNAME',
 'TEAM_ID',
 'TEAM_ABBREVIATION',
 'TEAM_NAME',
 'GAME_ID',
 'GAME_DATE',
 'MATCHUP',
 'WL',
 'MIN',
 'USG_PCT',
 'PCT_FGM',
 'PCT_FGA',
 'PCT_FG3M',
 'PCT_FG3A',
 'PCT_FTM',
 'PCT_FTA',
 'PCT_OREB',
 'PCT_DREB',
 'PCT_REB',
 'PCT_AST',
 'PCT_TOV',
 'PCT_STL',
 'PCT_BLK',
 'PCT_BLKA',
 'PCT_PF',
 'PCT_PFD',
 'PCT_PTS',
 'GP_RANK',
 'W_RANK',
 'L_RANK',
 'W_PCT_RANK',
 'MIN_RANK',
 'USG_PCT_RANK',
 'PCT_FGM_RANK',
 'PCT_FGA_RANK',
 'PCT_FG3M_RANK',
 'PCT_FG3A_RANK',
 'PCT_FTM_RANK',
 'PCT_FTA_RANK',
 'PCT_OREB_RANK',
 'PCT_DREB_RANK',
 'PCT_REB_RANK',
 'PCT_AST_RANK',
 'PCT_TOV_RANK',
 'PCT_STL_RANK',
 'PCT_BLK_RANK',
 'PCT_BLKA_RANK',
 'PCT_PF_RANK',
 'PCT_PFD_RANK',
 'PCT_PTS_RANK',
 'VIDEO_AVAILABLE_FLAG']

In [151]:
plyr_usage_df.head()

,SEASON_YEAR,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,...,PCT_REB_RANK,PCT_AST_RANK,PCT_TOV_RANK,PCT_STL_RANK,PCT_BLK_RANK,PCT_BLKA_RANK,PCT_PF_RANK,PCT_PFD_RANK,PCT_PTS_RANK,VIDEO_AVAILABLE_FLAG
0,2022-23,1629029,Luka Doncic,Luka,1610612742,DAL,Dallas Mavericks,0022200913,2023-02-26T00:00:00,DAL vs. LAL,...,37,49,24,15,20,30,16,46,38,1
1,2022-23,1629029,Luka Doncic,Luka,1610612742,DAL,Dallas Mavericks,0022200891,2023-02-23T00:00:00,DAL vs. SAS,...,24,10,6,26,4,1,1,23,36,1
2,2022-23,1629029,Luka Doncic,Luka,1610612742,DAL,Dallas Mavericks,0022200881,2023-02-15T00:00:00,DAL @ DEN,...,49,15,7,5,20,1,9,5,3,1
3,2022-23,1629029,Luka Doncic,Luka,1610612742,DAL,Dallas Mavericks,0022200865,2023-02-13T00:00:00,DAL vs. MIN,...,1,36,29,15,20,45,33,21,23,1
4,2022-23,1629029,Luka Doncic,Luka,1610612742,DAL,Dallas Mavericks,0022200854,2023-02-11T00:00:00,DAL @ SAC,...,37,51,34,43,20,1,1,41,47,1


In [152]:
# example NBA.com player index (https://stats.nba.com/stats/playergamelogs/playerindex):
# do we need anything...? Just get from bio?
plyrs_idx_url = 'https://stats.nba.com/stats/playerindex?College=&Country=&DraftPick=&DraftRound=&DraftYear=&Height=&Historical=1&LeagueID=00&Season=2022-23&SeasonType=Regular%20Season=2022-23&TeamID=0&Weight='
plyrs_idx_response = requests.get(url=plyrs_idx_url, headers=headers).json()
plyrs_idx_headers = plyrs_idx_response['resultSets'][0]['headers']
plyrs_idx_data = plyrs_idx_response['resultSets'][0]['rowSet']
plyrs_idx_df = pd.DataFrame(data=plyrs_idx_data, columns=plyrs_idx_headers)

In [153]:
plyrs_idx_headers # do we need anything...? Just get from bio?

['PERSON_ID',
 'PLAYER_LAST_NAME',
 'PLAYER_FIRST_NAME',
 'PLAYER_SLUG',
 'TEAM_ID',
 'TEAM_SLUG',
 'IS_DEFUNCT',
 'TEAM_CITY',
 'TEAM_NAME',
 'TEAM_ABBREVIATION',
 'JERSEY_NUMBER',
 'POSITION',
 'HEIGHT',
 'WEIGHT',
 'COLLEGE',
 'COUNTRY',
 'DRAFT_YEAR',
 'DRAFT_ROUND',
 'DRAFT_NUMBER',
 'ROSTER_STATUS',
 'PTS',
 'REB',
 'AST',
 'STATS_TIMEFRAME',
 'FROM_YEAR',
 'TO_YEAR']

In [154]:
plyrs_idx_df.head()

,PERSON_ID,PLAYER_LAST_NAME,PLAYER_FIRST_NAME,PLAYER_SLUG,TEAM_ID,TEAM_SLUG,IS_DEFUNCT,TEAM_CITY,TEAM_NAME,TEAM_ABBREVIATION,...,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,ROSTER_STATUS,PTS,REB,AST,STATS_TIMEFRAME,FROM_YEAR,TO_YEAR
0,76001,Abdelnaby,Alaa,alaa-abdelnaby,1610612757,blazers,0,Portland,Trail Blazers,POR,...,1990.0,1.0,25.0,NaN,5.7,3.3,0.3,Career,1990,1994
1,76002,Abdul-Aziz,Zaid,zaid-abdul-aziz,1610612745,rockets,0,Houston,Rockets,HOU,...,1968.0,1.0,5.0,NaN,9.0,8.0,1.2,Career,1968,1977
2,76003,Abdul-Jabbar,Kareem,kareem-abdul-jabbar,1610612747,lakers,0,Los Angeles,Lakers,LAL,...,1969.0,1.0,1.0,NaN,24.6,11.2,3.6,Career,1969,1988
3,51,Abdul-Rauf,Mahmoud,mahmoud-abdul-rauf,1610612743,nuggets,0,Denver,Nuggets,DEN,...,1990.0,1.0,3.0,NaN,14.6,1.9,3.5,Career,1990,2000
4,1505,Abdul-Wahad,Tariq,tariq-abdul-wahad,1610612758,kings,0,Sacramento,Kings,SAC,...,1997.0,1.0,11.0,NaN,7.8,3.3,1.1,Career,1997,2003


In [155]:
# example NBA.com 2022-23 (Season=2022-23) players bio (https://stats.nba.com/stats/leaguedashplayerbiostats):
# use for AGE, PLAYER_HEIGHT_INCHES, PLAYER_WEIGHT
plyrs_bio_url = 'https://stats.nba.com/stats/leaguedashplayerbiostats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PerMode=PerGame&Period=0&PlayerExperience=&PlayerPosition=&Season=2022-23&SeasonSegment=&SeasonType=Regular%20Season&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight='
plyrs_bio_response = requests.get(url=plyrs_bio_url, headers=headers).json()
plyrs_bio_headers = plyrs_bio_response['resultSets'][0]['headers']
plyrs_bio_data = plyrs_bio_response['resultSets'][0]['rowSet']
plyrs_bio_df = pd.DataFrame(data=plyrs_bio_data, columns=plyrs_bio_headers)

In [156]:
plyrs_bio_headers

['PLAYER_ID',
 'PLAYER_NAME',
 'TEAM_ID',
 'TEAM_ABBREVIATION',
 'AGE',
 'PLAYER_HEIGHT',
 'PLAYER_HEIGHT_INCHES',
 'PLAYER_WEIGHT',
 'COLLEGE',
 'COUNTRY',
 'DRAFT_YEAR',
 'DRAFT_ROUND',
 'DRAFT_NUMBER',
 'GP',
 'PTS',
 'REB',
 'AST',
 'NET_RATING',
 'OREB_PCT',
 'DREB_PCT',
 'USG_PCT',
 'TS_PCT',
 'AST_PCT']

In [157]:
plyrs_bio_df.head()

,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,AGE,PLAYER_HEIGHT,PLAYER_HEIGHT_INCHES,PLAYER_WEIGHT,COLLEGE,COUNTRY,...,GP,PTS,REB,AST,NET_RATING,OREB_PCT,DREB_PCT,USG_PCT,TS_PCT,AST_PCT
0,1630639,A.J. Lawson,1610612742,DAL,22.0,6-6,78,179,South Carolina,Canada,...,13,2.6,0.8,0.1,-10.9,0.028,0.151,0.182,0.622,0.029
1,1631260,AJ Green,1610612749,MIL,23.0,6-5,77,190,Northern Iowa,USA,...,27,4.7,1.1,0.6,-6.9,0.014,0.088,0.163,0.637,0.092
2,1631100,AJ Griffin,1610612737,ATL,19.0,6-6,78,220,Duke,USA,...,57,9.2,2.1,1.0,-0.5,0.022,0.076,0.173,0.583,0.068
3,203932,Aaron Gordon,1610612743,DEN,27.0,6-8,80,235,Arizona,USA,...,50,17.1,6.8,2.9,13.8,0.086,0.148,0.210,0.641,0.126
4,1628988,Aaron Holiday,1610612737,ATL,26.0,6-0,72,185,UCLA,USA,...,51,4.1,1.3,1.3,-0.4,0.027,0.062,0.128,0.527,0.115


In [322]:
plyr_box_stats_df = pd.concat([plyr_base_df_subset, 
                               plyr_advanced_df_subset, 
                               plyr_misc_df_subset, 
                               plyr_scor_df_subset], axis=1)

num_box_stats = plyr_box_stats_df.shape[0]

player_bio = plyrs_bio_df.loc[plyrs_bio_df['PLAYER_ID'] == player_id]

# cannot find player birthday info on NBA.com
# subtract age by one when going back single NBA season; this is the work-around for now
player_age = [int(player_bio.iloc[0]['AGE'])] * num_box_stats
plyr_box_stats_df['AGE'] = player_age

player_height = [player_bio.iloc[0]['PLAYER_HEIGHT_INCHES']] * num_box_stats
plyr_box_stats_df['PLAYER_HEIGHT_INCHES'] = player_height

player_weight = [player_bio.iloc[0]['PLAYER_WEIGHT']] * num_box_stats
plyr_box_stats_df['PLAYER_WEIGHT'] = player_weight

In [323]:
fg_attr = ['FGM','FGA','FG3M','FG3A','FTM','FTA','PTS','PTS_PAINT','PCT_PTS_2PT_MR']

fgm_paint_list = []
fgm_mr_list = []
for x in range(0, num_box_stats):
    game_stats = plyr_box_stats_df.iloc[x][fg_attr]
    fgm_paint = game_stats['PTS_PAINT']/2
    fgm_2p = game_stats['FGM'] - game_stats['FG3M']
    fgm_mr = fgm_2p - fgm_paint
    fgm_paint_list.append(int(fgm_paint))
    fgm_mr_list.append(int(fgm_mr))

# note: cannot find 2PT Mid-range FG attempts in player box scores at NBA.com;
# will use 2PT mid-range and in-the-paint made FG for now
plyr_box_stats_df['FGM_PAINT'] = fgm_paint_list
plyr_box_stats_df['FGM_2PT_MR'] = fgm_mr_list


In [324]:
plyr_box_stats_df

,SEASON_YEAR,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,...,USG_PCT,PIE,POSS,PTS_PAINT,PCT_PTS_2PT_MR,AGE,PLAYER_HEIGHT_INCHES,PLAYER_WEIGHT,FGM_PAINT,FGM_2PT_MR
0,2022-23,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022200913,2023-02-26T00:00:00,DAL vs. LAL,L,...,0.319,0.140,78,10,0.077,24,79,230,5,1
1,2022-23,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022200891,2023-02-23T00:00:00,DAL vs. SAS,W,...,0.319,0.264,65,14,0.000,24,79,230,7,0
2,2022-23,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022200881,2023-02-15T00:00:00,DAL @ DEN,L,...,0.359,0.236,72,12,0.162,24,79,230,6,3
3,2022-23,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022200865,2023-02-13T00:00:00,DAL vs. MIN,L,...,0.378,0.202,73,16,0.000,24,79,230,8,0
4,2022-23,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022200854,2023-02-11T00:00:00,DAL @ SAC,L,...,0.304,0.121,85,16,0.074,24,79,230,8,1
5,2022-23,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022200782,2023-02-02T00:00:00,DAL vs. NOP,W,...,0.500,0.301,49,18,0.065,24,79,230,9,1
6,2022-23,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022200761,2023-01-30T00:00:00,DAL vs. DET,W,...,0.439,0.394,74,20,0.075,24,79,230,10,2
7,2022-23,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022200734,2023-01-26T00:00:00,DAL @ PHX,W,...,0.222,-0.039,7,0,0.000,24,79,230,0,0
8,2022-23,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022200717,2023-01-24T00:00:00,DAL vs. WAS,L,...,0.419,0.234,81,26,0.146,24,79,230,13,3
9,2022-23,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022200701,2023-01-22T00:00:00,DAL vs. LAC,L,...,0.383,0.158,70,10,0.069,24,79,230,5,1


In [325]:
def get_games_past_week(this_game, plyr_box_stats_df):
    # for a given game_date find the week period before
    game_date_str = this_game['GAME_DATE']
    # parse game_date into python date object
    game_date = datetime.datetime.strptime(game_date_str, '%Y-%m-%dT%H:%M:%S')
    time_delta_yesterday = datetime.timedelta(days = 1)
    time_delta_week_ago = datetime.timedelta(days = 7)
    date_yesterday = game_date - time_delta_yesterday
    date_week_ago = game_date - time_delta_week_ago

    # convert game_date to python date objects
    game_dates = plyr_box_stats_df['GAME_DATE'].values.tolist()
    game_dates_obj = [datetime.datetime.strptime(date, '%Y-%m-%dT%H:%M:%S') for date in game_dates]

    # create temp df of only game_id and game_date
    game_dates_df = plyr_box_stats_df[['GAME_ID','GAME_DATE']]

    # add converted python game_date object to temp df
    game_dates_df.loc[:,'GAME_DATE_OBJ']  = game_dates_obj
    
    # find games that lie between date_yesterday and date_week_ago
    game_dates_past_week = game_dates_df.loc[(game_dates_df['GAME_DATE_OBJ'] <= date_yesterday) & (game_dates_df['GAME_DATE_OBJ'] >= date_week_ago)]
    game_ids_past_week = game_dates_past_week['GAME_ID'].values.tolist()
    games_past_week = plyr_box_stats_df.loc[plyr_box_stats_df['GAME_ID'].isin(game_ids_past_week)]
    
    return games_past_week

In [326]:
'''
Compile box stats containing stat totals from the past week (7 days before) of game_date for the following: 
Total minutes played - pw_min
Total games played - pw_games
2-point field goal attempts - pw_fg2a
3-point field goal attempts - pw_gf3a
Blocks - pw_blk
Rebounds - pw_reb (need offensive/defensive rebounds…?)
Personal fouls - pw_pf
Assists - pw_ast
'''

'\nCompile box stats containing stat totals from the past week (7 days before) of game_date for the following: \nTotal minutes played - pw_min\nTotal games played - pw_games\n2-point field goal attempts - pw_fg2a\n3-point field goal attempts - pw_gf3a\nBlocks - pw_blk\nRebounds - pw_reb (need offensive/defensive rebounds…?)\nPersonal fouls - pw_pf\nAssists - pw_ast\n'

In [327]:
pw_mins = []
pw_gamess = []
pw_fg3as = []
pw_fg2as = []
pw_orebs = []
pw_drebs = []
pw_asts = []
pw_tovs = []
pw_stls = []
pw_blks = []
pw_posss = []
pw_pfs = []
pw_pfds = []
pw_pts_paints = []
pw_pts_2pt_mrs = []
pw_pie_avg = []
pw_usg_pct_avg = []

In [328]:
# iterate through all games in player box stats and calculate totals/avgs for games the week before
for index, this_game in plyr_box_stats_df.iterrows():
    games_week_before_df = get_games_week_before(this_game, plyr_box_stats_df)
    pw_min = games_week_before_df['MIN'].sum()
    pw_games = games_week_before_df.shape[0]
    pw_fg3a = games_week_before_df['FG3A'].sum()
    pw_fg2a = games_week_before_df['FGA'].sum() - pw_fg3a
    pw_oreb = games_week_before_df['OREB'].sum()
    pw_dreb = games_week_before_df['DREB'].sum()
    pw_ast = games_week_before_df['AST'].sum()
    pw_tov = games_week_before_df['TOV'].sum()
    pw_stl = games_week_before_df['STL'].sum()
    pw_blk = games_week_before_df['BLK'].sum()
    pw_poss = games_week_before_df['POSS'].sum()
    pw_pf = games_week_before_df['PF'].sum()
    pw_pfd = games_week_before_df['PFD'].mean()
    pw_pts_paint = games_week_before_df['PTS_PAINT'].sum()
    pw_pts_2pt_mr = games_week_before_df['PCT_PTS_2PT_MR'].sum()
    pw_pie = games_week_before_df['PIE'].mean()
    pw_usg_pct = games_week_before_df['USG_PCT'].mean()
    
    pw_mins.append(pw_min)
    pw_gamess.append(pw_games)
    pw_fg3as.append(pw_fg3a)
    pw_fg2as.append(pw_fg2a)
    pw_orebs.append(pw_oreb)
    pw_drebs.append(pw_dreb)
    pw_asts.append(pw_ast)
    pw_tovs.append(pw_tov)
    pw_stls.append(pw_stl)
    pw_blks.append(pw_blk)
    pw_posss.append(pw_poss)
    pw_pfs.append(pw_pf)
    pw_pfds.append(pw_pfd)
    pw_pts_paints.append(pw_pts_paint)
    pw_pts_2pt_mrs.append(pw_pts_2pt_mr)
    pw_pie_avg.append(pw_pie)
    pw_usg_pct_avg.append(pw_usg_pct)

C:\Users\Reed\AppData\Local\Temp\ipykernel_7540\3095134818.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_dates_df.loc[:,'GAME_DATE_OBJ']  = game_dates_obj
C:\Users\Reed\AppData\Local\Temp\ipykernel_7540\3095134818.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_dates_df.loc[:,'GAME_DATE_OBJ']  = game_dates_obj
C:\Users\Reed\AppData\Local\Temp\ipykernel_7540\3095134818.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

C:\Users\Reed\AppData\Local\Temp\ipykernel_7540\3095134818.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_dates_df.loc[:,'GAME_DATE_OBJ']  = game_dates_obj
C:\Users\Reed\AppData\Local\Temp\ipykernel_7540\3095134818.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_dates_df.loc[:,'GAME_DATE_OBJ']  = game_dates_obj
C:\Users\Reed\AppData\Local\Temp\ipykernel_7540\3095134818.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [329]:
plyr_box_stats_df.loc[:,'PW_MIN'] = pw_mins
plyr_box_stats_df.loc[:,'PW_GAMES'] = pw_gamess
plyr_box_stats_df.loc[:,'PW_GAMES'] = pw_gamess
plyr_box_stats_df.loc[:,'PW_FG3A'] = pw_fg3as
plyr_box_stats_df.loc[:,'PW_FG2A'] = pw_fg2as
plyr_box_stats_df.loc[:,'PW_OREB'] = pw_orebs
plyr_box_stats_df.loc[:,'PW_DREB'] = pw_drebs
plyr_box_stats_df.loc[:,'PW_ASTS'] = pw_asts
plyr_box_stats_df.loc[:,'PW_TOV'] = pw_tovs
plyr_box_stats_df.loc[:,'PW_STL'] = pw_stls
plyr_box_stats_df.loc[:,'PW_BLK'] = pw_blks
plyr_box_stats_df.loc[:,'PW_POS'] = pw_posss
plyr_box_stats_df.loc[:,'PW_PF'] = pw_pfs
plyr_box_stats_df.loc[:,'PW_PFD'] = pw_pfds
plyr_box_stats_df.loc[:,'PW_PTS_PAINT'] = pw_pts_paints
plyr_box_stats_df.loc[:,'PW_PTS_2PT_MR'] = pw_pts_2pt_mrs
plyr_box_stats_df.loc[:,'PW_PIE_AVG'] = pw_pie_avg
plyr_box_stats_df.loc[:,'PW_USG_PCT_AVG'] = pw_usg_pct_avg

# fill na with 0 on past week stats
plyr_box_stats_df.loc[:,'PW_PFD'] = plyr_box_stats_df['PW_PFD'].fillna(0)
plyr_box_stats_df.loc[:,'PW_PIE_AVG'] = plyr_box_stats_df['PW_PIE_AVG'].fillna(0)
plyr_box_stats_df.loc[:,'PW_USG_PCT_AVG'] = plyr_box_stats_df['PW_USG_PCT_AVG'].fillna(0)

In [330]:
plyr_box_stats_df

,SEASON_YEAR,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,...,PW_TOV,PW_STL,PW_BLK,PW_POS,PW_PF,PW_PFD,PW_PTS_PAINT,PW_PTS_2PT_MR,PW_PIE_AVG,PW_USG_PCT_AVG
0,2022-23,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022200913,2023-02-26T00:00:00,DAL vs. LAL,L,...,1,2,2,65,0,7.000000,14,0.000,0.264000,0.319000
1,2022-23,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022200891,2023-02-23T00:00:00,DAL vs. SAS,W,...,0,0,0,0,0,0.000000,0,0.000,0.000000,0.000000
2,2022-23,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022200881,2023-02-15T00:00:00,DAL @ DEN,L,...,10,1,0,158,3,7.000000,32,0.074,0.161500,0.341000
3,2022-23,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022200865,2023-02-13T00:00:00,DAL vs. MIN,L,...,6,0,0,85,0,4.000000,16,0.074,0.121000,0.304000
4,2022-23,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022200854,2023-02-11T00:00:00,DAL @ SAC,L,...,0,0,0,0,0,0.000000,0,0.000,0.000000,0.000000
5,2022-23,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022200782,2023-02-02T00:00:00,DAL vs. NOP,W,...,4,2,0,81,2,7.000000,20,0.075,0.177500,0.330500
6,2022-23,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022200761,2023-01-30T00:00:00,DAL vs. DET,W,...,3,1,0,88,3,4.500000,26,0.146,0.097500,0.320500
7,2022-23,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022200734,2023-01-26T00:00:00,DAL @ PHX,W,...,10,3,0,220,6,8.666667,48,0.274,0.231333,0.389667
8,2022-23,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022200717,2023-01-24T00:00:00,DAL vs. WAS,L,...,14,2,0,222,6,8.000000,34,0.195,0.187000,0.358667
9,2022-23,1629029,Luka Doncic,1610612742,DAL,Dallas Mavericks,0022200701,2023-01-22T00:00:00,DAL vs. LAC,L,...,10,1,0,152,3,7.500000,24,0.126,0.201500,0.346500


In [301]:
plyr_box_stats_df.columns

Index(['SEASON_YEAR', 'PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID',
       'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID', 'GAME_DATE', 'MATCHUP',
       'WL', 'MIN', 'FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 'OREB', 'DREB',
       'AST', 'TOV', 'STL', 'BLK', 'PF', 'PFD', 'PTS', 'USG_PCT', 'PIE',
       'POSS', 'PTS_PAINT', 'PCT_PTS_2PT_MR', 'AGE', 'PLAYER_HEIGHT_INCHES',
       'PLAYER_WEIGHT', 'FGM_PAINT', 'FGM_2PT_MR', 'PW_MIN', 'PW_GAMES',
       'PW_FG3A', 'PW_FG2A', 'PW_OREB', 'PW_DREB', 'PW_ASTS', 'PW_TOV',
       'PW_STL', 'PW_BLK', 'PW_POS', 'PW_PF', 'PW_PFD', 'PW_PTS_PAINT',
       'PW_PTS_2PT_MR', 'PW_PIE_AVG', 'PW_USG_PCT_AVG'],
      dtype='object')

In [306]:
plyr_box_stats_df.to_csv('Luka_Doncic_1629029_2022_23_game_logs.csv')

In [ ]:
# to-do: work on injury stats